In [1]:
%cd ../..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
import os.path as op
import time

from fastmri_recon.config import *
from fastmri_recon.data.fastmri_tf_datasets import train_masked_kspace_dataset_from_indexable
from fastmri_recon.helpers.nn_mri import tf_fastmri_format
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tqdm.notebook import tqdm

from learning_wavelets.keras_utils.fourier import tf_masked_shifted_normed_fft2d, tf_masked_shifted_normed_ifft2d
from learning_wavelets.models.ista import IstaLearnlet

In [4]:
# paths
val_path = f'{FASTMRI_DATA_DIR}singlecoil_val/'

n_volumes_train = 973
af = 4
val_set = train_masked_kspace_dataset_from_indexable(
    val_path,
    AF=af,
    contrast='CORPD_FBK',  # ['CORPDFS_FBK', 'CORPD_FBK', None]
    scale_factor=1e6,
)

Getting training files from /media/Zaccharie/UHRes/singlecoil_val/


In [5]:
learnlet_params = {
    'denoising_activation': 'dynamic_soft_thresholding',
    'learnlet_analysis_kwargs':{
        'n_tiling': 16, 
        'mixing_details': False,    
        'skip_connection': True,
        'kernel_size': 7,
    },
    'learnlet_synthesis_kwargs': {
        'res': True,
        'kernel_size': 9,
    },
    'threshold_kwargs':{
        'noise_std_norm': False,
        'alpha_init': 2.0,
    },
    'n_scales': 4,
    'n_reweights_learn': 1,
    'exact_reconstruction': True,
    'undecimated': True,
    'clip': False,
}

model = IstaLearnlet(
    n_iterations=5,
    forward_operator=tf_masked_shifted_normed_fft2d,
    adjoint_operator=tf_masked_shifted_normed_ifft2d,
    postprocess=tf_fastmri_format,
    fista_mode=True,
    **learnlet_params,
)
model([tf.zeros((1, 640, 320, 1), dtype=tf.complex64), tf.zeros((1, 640, 320), dtype=tf.complex64)])
model.load_weights(
    'checkpoints/fista_learnlet_fastmri_16_5_dynamic_soft_thresholding_1583847877-300.hdf5',
)

In [8]:
n_samples = 100
psnrs = tf.TensorArray(tf.float32, size=n_samples)
i = 0
for kspace_mask, image in tqdm(val_set.take(n_samples)):
    res = tf.convert_to_tensor(model.predict(kspace_mask, batch_size=1))
    psnrs = psnrs.write(i, tf.reduce_mean(tf.image.psnr(res[..., 0][None, ...], image[..., 0][None, ...], max_val=tf.reduce_max(image))))
    i += 1
tf.reduce_mean(psnrs.stack())

<tf.Tensor: shape=(), dtype=float32, numpy=33.316612>